In [2]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import time

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [3]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [5]:
bigbrotr.connect()

In [6]:
query = "SELECT * FROM relay_metadata"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

('wss://relay.mostr.pub', 1747575335, True, True, True, True, False, 1323, 168, 189, 'Mostr relay', 'Relay to bridge Nostr with ActivityPub.', None, None, None, None, None, 'git+https://github.com/hoytech/strfry.git', '1.0.4', None, None, {'max_limit': 500, 'max_subscriptions': 20, 'max_message_length': 131072}, {'negentropy': 1})
('wss://relay.damus.io', 1747575335, True, True, True, True, True, 272, 185, 212, 'damus.io', 'Damus strfry relay', None, 'https://damus.io/img/logo.png', '32e1827635450ebb3c5a7d12c1f8e7b2b514439ac10a67eef3d9fd9c5c68e245', 'jb55@jb55.com', None, 'git+https://github.com/hoytech/strfry.git', '1.0.4-1-g783f9ce8cc77', None, None, {'max_limit': 500, 'max_subscriptions': 300, 'max_message_length': 1000000}, {'negentropy': 1})
('wss://nos.lol', 1747575335, True, True, True, True, True, 92, 29, 53, 'nos.lol', 'Generally accepts notes, except spammy ones.', None, None, 'c5fadeb5d90d68baffc631455a07ca340ccf1e31110955e16d45eb5f87147cd9', 'https://wikifreedia.xyz/nos.lol

In [ ]:
bigbrotr.close()

# relay_urls

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)

# compute_relay_metadata

In [ ]:
from relay import Relay
from aiohttp import ClientSession

relay = Relay('wss://puravida.nostr.land')

async def fetch_nip11_metadata(relay_id, session, timeout):
    headers = {'Accept': 'application/nostr+json'}
    for schema in ['https://', 'http://']:
        try:
            async with session.get(schema + relay_id, headers=headers, timeout=timeout) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    pass
        except Exception as e:
            pass
    return None

relay_id = relay.url.removeprefix('wss://')
nip11_raw = None
async with ClientSession() as session:
    nip11_raw = await fetch_nip11_metadata(relay_id, session, 10)
nip11_raw

In [9]:
from compute_relay_metadata import compute_relay_metadata

sec, pub = utils.generate_nostr_keypair()
metadata = await compute_relay_metadata(relay, sec, pub, socks5_proxy_url=None, timeout=10)
metadata.to_dict()

fetch_nip11_metadata error (saltivka.org): 200, message='Attempt to decode JSON with unexpected mimetype: text/html; charset=utf-8', url='http://ww25.saltivka.org/?subid1=20250519-0136-0582-98fd-696f4d30c962'
fetch_nip11_metadata error (saltivka.org): 200, message='Attempt to decode JSON with unexpected mimetype: text/html; charset=utf-8', url='http://ww25.saltivka.org/?subid1=20250519-0136-05de-ba91-3061efae250b'
check_connectivity error (wss://saltivka.org): 200, message='Invalid response status', url='http://ww25.saltivka.org/?subid1=20250519-0136-06ef-896b-7130d984067d'
check_connectivity error (ws://saltivka.org): 200, message='Invalid response status', url='http://ww25.saltivka.org/?subid1=20250519-0136-06cd-8e7b-e33790e81535'


{'relay': Relay(url=wss://saltivka.org, network=clearnet),
 'generated_at': 1747582566,
 'connection_success': False,
 'nip11_success': False,
 'openable': None,
 'readable': None,
 'writable': None,
 'rtt_open': None,
 'rtt_read': None,
 'rtt_write': None,
 'name': None,
 'description': None,
 'banner': None,
 'icon': None,
 'pubkey': None,
 'contact': None,
 'supported_nips': None,
 'software': None,
 'version': None,
 'privacy_policy': None,
 'terms_of_service': None,
 'limitation': None,
 'extra_fields': None}

In [8]:
relay = Relay('wss://saltivka.org')
